<a href="https://colab.research.google.com/github/ambideXtrous9/NQA-BiLSTM/blob/main/NQA_Model_MHA%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.6 MB/s eta 0:00:00


In [1]:
import torch
import pytorch_lightning as pl
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [2]:
pl.seed_everything (42)


INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'


In [5]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [6]:
df = df.iloc[:5000]

In [7]:
class NQADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        question = row['question']
        context = row['paragraph']
        answer = row['answer']
        
        input_ids = self.tokenizer(question, 
                                    context,
                                    padding = "max_length",
                                    truncation = "only_second",
                                    return_attention_mask = True,
                                    add_special_tokens = True,
                                    return_tensors = "pt").input_ids
        
        answer_start = context.find(answer)
        answer_end = answer_start + len(answer) - 1
   
        return dict(input_ids = input_ids.flatten(), 
                    answer_start=torch.tensor(answer_start),
                    answer_end=torch.tensor(answer_end))

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
sample_dataset = NQADataset(df,tokenizer)

In [10]:
for data in sample_dataset:
  print(data['input_ids'].size())
  print(data['answer_start'])
  print(data['answer_end'])

  break

torch.Size([512])
tensor(134)
tensor(145)


In [11]:
train_df, val_df = train_test_split(df,test_size=0.2)
val_df, test_df = train_test_split(val_df,test_size=0.5)

In [12]:
class NQADataModule(pl.LightningDataModule):
  def __init__(self,train_df , val_df, test_df,tokenizer,batch_size : int = 8):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer
    

  def setup(self,stage=None):
    self.train_dataset = NQADataset(self.train_df,self.tokenizer,)
    self.val_dataset = NQADataset(self.val_df,self.tokenizer)
    self.test_dataset = NQADataset(self.test_df,self.tokenizer)
    

  def train_dataloader(self):
    return DataLoader(self.train_dataset,batch_size = self.batch_size,shuffle=True,num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.val_dataset,batch_size = self.batch_size,num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size = self.batch_size,num_workers=2)

In [13]:
BATCH_SIZE = 64
N_EPOCHS = 20

data_module = NQADataModule(train_df,val_df,test_df,tokenizer,batch_size = BATCH_SIZE)
data_module.setup()

In [14]:
class NQAModel(pl.LightningModule):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout,num_heads):
        super().__init__()
        
        # Embedding layer
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        
        # Bi-LSTM layer
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)
        
        # Multihead attention layer
        self.num_heads = num_heads
        self.attention = torch.nn.MultiheadAttention(hidden_size * 2, num_heads=num_heads)


        # Output layer
        self.linear_start = torch.nn.Linear(hidden_size * 2, 1)
        self.linear_end = torch.nn.Linear(hidden_size * 2, 1)
        
    def forward(self, input_ids):
        # Embedding layer
        embeddings = self.embedding(input_ids)
        
        # Bi-LSTM layer
        lstm_output, _ = self.lstm(embeddings)
        
        # Multihead attention layer
        attention_output, _ = self.attention(lstm_output.transpose(0, 1), lstm_output.transpose(0, 1), lstm_output.transpose(0, 1))
        attention_output = attention_output.transpose(0, 1)
        
        # Output layer
        start_logits = self.linear_start(attention_output).squeeze(-1)
        end_logits = self.linear_end(attention_output).squeeze(-1)
        
        return start_logits, end_logits
    
    def training_step(self, batch, batch_idx):
        # Training step
        input_ids = batch['input_ids']
        answer_start = batch['answer_start']
        answer_end = batch['answer_end']
        start_logits, end_logits = self(input_ids)
        loss_start = torch.nn.functional.cross_entropy(start_logits, answer_start)
        loss_end = torch.nn.functional.cross_entropy(end_logits, answer_end)
        loss = loss_start + loss_end
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # Validation step
        input_ids = batch['input_ids']
        answer_start = batch['answer_start']
        answer_end = batch['answer_end']
        start_logits, end_logits = self(input_ids)
        loss_start = torch.nn.functional.cross_entropy(start_logits, answer_start)
        loss_end = torch.nn.functional.cross_entropy(end_logits, answer_end)
        loss = loss_start + loss_end
        self.log('val_loss', loss)
    
    def configure_optimizers(self):
        # Optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

In [15]:
model = NQAModel(vocab_size=len(tokenizer.vocab), embedding_dim=128, hidden_size=256, num_layers=4, dropout=0.2,num_heads=4)


In [16]:
checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'best_cp',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min'
)

In [17]:
trainer = pl.Trainer(devices=-1, accelerator="gpu",
    callbacks=[checkpoint_callback],
    max_epochs = N_EPOCHS
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model,data_module)


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type               | Params
----------------------------------------------------
0 | embedding    | Embedding          | 3.9 M 
1 | lstm         | LSTM               | 5.5 M 
2 | attention    | MultiheadAttention | 1.1 M 
3 | linear_start | Linear             | 513   
4 | linear_end   | Linear             | 513   
----------------------------------------------------
10.5 M    Trainable params
0         Non-trainable params
10.5 M    Total params
41.919    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
def predict_single(test_df, idx, model, tokenizer):
    model.eval()
    question = test_df.iloc[idx]['question']
    context = test_df.iloc[idx]['paragraph']
    answer = test_df.iloc[idx]['answer']

    print("Actual Answer : ",answer)

    input_ids = tokenizer(question, 
                          context,
                          padding="max_length",
                          truncation="only_second",
                          return_attention_mask=True,
                          add_special_tokens=True,
                          return_tensors="pt").input_ids  # Add a new dimension

    start_logits, end_logits = model(input_ids)
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()
    ans = tokenizer.decode(input_ids[0][start_index:end_index+1], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print("Pred Answer : ", ans)


In [ ]:
predict_single(test_df,8,model,tokenizer)